## Import Datasets

**Make sure that you've downloaded the required human and dog datasets:
Download the dog dataset. Unzip the folder and place it in this project's home directory, at the location /dog_images.
Download the human dataset. Unzip the folder and place it in the home directory, at location /lfw.**

In [ ]:
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/lfw.zip

In [ ]:
#!unzip dogImages.zip
#!unzip lfw.zip

In [ ]:
from glob import glob

# load filenames for human and dog images
human_files = np.array(glob("./lfw/*/*"))
dog_files = np.array(glob("./dogImages/*/*/*"))

# print number of images in each dataset
print('There are %d total human images.' % len(human_files))
print('There are %d total dog images.' % len(dog_files))

In [ ]:
from PIL import Image
image = Image.open(dog_files[1])
plt.imshow(image)

## Obtain Pre-trained VGG-16 Model
**The code cell below downloads the VGG-16 model, along with weights that have been trained on ImageNet, a very large, very popular dataset used for image classification and other vision tasks. ImageNet contains over 10 million URLs, each linking to an image containing an object from one of 1000 categories.**

In [ ]:
from keras.applications.vgg16 import VGG16
model = VGG16()

In [ ]:
model.summary()

First, we can use the load_img() function to load the image and resize it to the required size of 224×224 pixels.

In [ ]:
from keras.preprocessing.image import load_img

image = load_img('./dogImages/train/001.Affenpinscher/Affenpinscher_00002.jpg',target_size=(224, 224))

Next, we can convert the pixels to a NumPy array so that we can work with it in Keras. We can use the img_to_array() function for this. We only have one sample (one image). We can reshape the array by calling reshape() and adding the extra dimension.

In [ ]:
from keras.preprocessing.image import img_to_array
# convert the image pixels to a numpy array
image = img_to_array(image)
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

Keras provides a function called preprocess_input() to prepare new input for the network.



We are now ready to make a prediction for our loaded and prepared image.



In [ ]:
yhat = model.predict(image)
print(yhat)

In [ ]:
from keras.applications.vgg16 import decode_predictions

label = decode_predictions(yhat)
print(label)

In [ ]:
label = label[0][0]
print(label)

In [ ]:
print('%s (%.2f%%)' % (label[1], label[2]*100))

# Complite solution

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
# load the model
model = VGG16()
# load an image from file
image = load_img('./dogImages/train/001.Affenpinscher/Affenpinscher_00002.jpg',target_size=(224, 224))
# convert the image pixels to a numpy array
image = img_to_array(image)
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# prepare the image for the VGG model
image = preprocess_input(image)
# predict the probability across all output classes
yhat = model.predict(image)
# convert the probabilities to class labels
label = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = label[0][0]
# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))

# Now we do transfer learning

In [ ]:
import os
base_dir = '/content/drive/My Drive/Colab Notebooks/dogImages'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
validation_dir = os.path.join(base_dir, 'valid')

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Very Important #
# In this way we use the folders name as lables of different breads#

training_set = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=32,
                                             image_size=(150, 150))

test_dataset = image_dataset_from_directory(test_dir,
                                                  shuffle=True,
                                                  batch_size=32,
                                                  image_size=(150, 150))

val_dataset = image_dataset_from_directory(validation_dir,
                                                  shuffle=True,
                                                  batch_size=32,
                                                  image_size=(150, 150))

# Detailed setting about lables_mode and etc ... can be found in the following link
# https://docs.w3cub.com/tensorflow~2.3/keras/preprocessing/image_dataset_from_directory

In [ ]:
data_augmentation = keras.Sequential(
    [       keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
   keras.layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
for images, labels in training_set.take(1):
    plt.figure(figsize=(12, 12))
    first_image = images[0]
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1)
        augmented_image = data_augmentation(
            tf.expand_dims(first_image, 0)
        )
        plt.imshow(augmented_image[0].numpy().astype("int32"))
        plt.axis("off")

In [ ]:
for images, labels in training_set.take(3):
print(labels)

In [ ]:
# visualizing different breads with their labels specified
plt.figure(figsize=(10, 10))
for images, labels in training_set.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

In [ ]:
base_model = keras.applications.Xception(
    weights='imagenet',  
    input_shape=(150, 150, 3),
    include_top=False) 

In [ ]:
base_model.trainable = False

In [ ]:
inputs = keras.Input(shape=(150, 150, 3))


In [ ]:
x = data_augmentation(inputs) 
x = tf.keras.applications.xception.preprocess_input(x)
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  
outputs = keras.layers.Dense(133, activation='softmax')(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy' ,metrics=keras.metrics.BinaryAccuracy())
model.fit(training_set, epochs=5, validation_data=val_dataset)